In [1]:
import gudhi
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import gudhi.representations
from itertools import combinations
import seaborn as sns
from gudhi.hera import wasserstein_distance
from gudhi.hera import bottleneck_distance
from collections import defaultdict
from scipy.interpolate import interp1d
from scipy.stats import ks_2samp

In [2]:
def nodesExtractor(name): #extracts nodes and their corresponding information
    file_path = 'Complement/' + name +'.csv'
    # Extract the 'connectivity' field from the 'Data' structured array
    points = pd.read_csv(file_path)
    points = points.iloc[:, 1:]
    # Save the DataFrame to inspect it
    return points

In [ ]:
pressure = '4'
lobe = 'left'

datasets = {
    'm1053007': nodesToArrayH('m1p'+pressure+'_053007_'+lobe),
    'm2053007': nodesToArrayH('m2p'+pressure+'_053007_'+lobe),
    'm1053107': nodesToArrayH('m1p'+pressure+'_053107_'+lobe),
    'm2053107': nodesToArrayH('m2p'+pressure+'_053107_'+lobe),
    'm1060107': nodesToArrayH('m1p'+pressure+'_060107_'+lobe),
    'm1060407': nodesToArrayC('m1p'+pressure+'_060407_'+lobe),
    'm2060407': nodesToArrayC('m2p'+pressure+'_060407_'+lobe),
    'm3060407': nodesToArrayC('m3p'+pressure+'_060407_'+lobe),
    'm1060507': nodesToArrayC('m1p'+pressure+'_060507_'+lobe),
    'm2060507': nodesToArrayC('m2p'+pressure+'_060507_'+lobe),
    'm3060507': nodesToArrayC('m3p'+pressure+'_060507_'+lobe),
    'm2060607': nodesToArrayC('m2p'+pressure+'_060607_'+lobe),
    'm3060607': nodesToArrayC('m3p'+pressure+'_060607_'+lobe),
}